# Combining our tools with pandas

This is a quick assignment meant to show you how to make pandas work together with other tools.

* **SQL** is great for large datasets
* **Python** is great for poking around at data
* **Command line tools** are great for quick analysis
* **Excel** is easily shareable

### Before we start

Import pandas the right way.

## Postgres to pandas: using `read_sql` in pandas

Hm, problem 2 of one of your SQL assignments seems interesting.

> Using avg(), get the average solar energy usage for each country, and display only the countries with an average that is above 500 (million). Make sure it is properly ordered like the results below, and wrap your avg column in round() to get round numbers:
> 
>
> |avg solar energy|country|
> |---|---|
> |91339.0|China|
> |20425.0|India|
> |14084.0|United States|
> |12142.0|Germany|
> |...|...|

Use `pd.read_sql` to run the query against your Postgres database and return the result as a pandas dataframe. While you can use `pg8000`, I think SQLAlchemy is the more common method for connecting to a database.

* **TIP:**  [This page might help](https://towardsdatascience.com/work-with-sql-in-python-using-sqlalchemy-and-pandas-cd7693def708)
* **TIP:** If you didn't get the answer to that homework, just.... run anything, I guess.

## CSV to SQLite: using `to_sql` in pandas

SQLite is like a baby version of Postgres. It's super convenient because it just creates a file instead of needing a big giant server!

Even if you used `pg8000` for the last one, you should probably level up to SQLAlchemy. [Here are notes on connections](https://docs.sqlalchemy.org/en/13/core/connections.html), and [here's how `.to_sql` works](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_sql.html).

In [82]:
# !pip install sqlalchemy

### Step 1: Create a SQLAlchemy "engine" (a connection) for a SQLite database you'll call `world.db`

* **TIP:** The filename format for a sqlite connection is `sqlite:///filename.db` – that's *three forward slashes*. More and you'll get in trouble.

### Step 2: Read `countries.csv` and `continents.csv` in as separate dataframes

### Step 3: Using `.to_sql`

Using your SQLAlchemy connection to `world.db`, save the countries data to a table called "countries" and the continent data to a table called "continents"

**If the table already exists, it should be replaced.**

* **TIP:** As stated in the headline, you will be using `.to_sql`
* **TIP:** Use `index=False` when saving, or else the index will for some reason get its own column.
* **TIP:** Shift+tab will give you documentation! The connection parameter is `con=` and not `conn=` because nothing in pandas can ever be nice.

### Step 4: Using `.read_sql`

Use `pd.read_sql` and your sqlite engine to run a query on your database and save the result into a pandas dataframe. I've given you your query below:

```python
query = """
SELECT * FROM countries
JOIN continents ON countries.continent = continents.continent
"""
```

The result should be a dataframe that includes the columns country, continent, life expectancy, population, pdf, continent, and square miles. It should only have ONE index, on the far left hand side, bolded. If it has multiples then you need to add `index=False` to the previous commands.

Why sqlite? ...because I don't have postgres installed here and I don't want to accidentally set something up wrong.

## CSVkit to SQLite

I dunno, that seemed like a lot of work. Can't I just use like CSVKit to something to shove all the data into a database?

Using [this as reference](https://gist.github.com/martijngastkemper/6d62245565a49081db6024f5f432fe63), use the `csvsql` query to insert the data from `worldcities.csv` into `world.db`. Have it insert into a table called `cities`, and overwrite the table if it already exists.

Test if it worked by successfully running the following using `.read_sql`.

```
SELECT
    cities.city, countries.country, cities.population,
    ROUND(cities.population / countries.population * 100) AS pct_pop
FROM cities
JOIN countries ON cities.country = countries.country
ORDER BY pct_pop DESC
LIMIT 10
```

It should calculate percent of the country's population each city is, but it... the data is bad. It's a data issue.

## Reading Google Sheets from pandas

I was trying to find a good spreadsheet for you to look at, and for some reason sitting at the top of my Google Drive was [this random list of anime and various scores and things](https://docs.google.com/spreadsheets/d/1CV9_bm7LrrjlDNb12ftBZBolAhtfunZP8j9g1sxbDFM/edit#gid=77782877). Why? I have no idea. Seems official, though.

### Read in the `Most favorited` spreadsheet and look at the first five

> **TIP:** [Use this walkthrough](https://towardsdatascience.com/read-data-from-google-sheets-into-pandas-without-the-google-sheets-api-5c468536550), but beware of fancy quotes

### Read in the `Most Dropped` spreadsheet and look at the first 5

What was the difference between the two? The URL is *mostly* the same.

## Reading HTML tables with pandas

This is the real killer app here. `pd.read_html` doesn't really read HTML, it reads *HTML tables*.

Use `pd.read_html` to read in the big watermelons from [here](http://www.bigpumpkins.com/WeighoffResultsGPC.aspx?c=W&y=2022). Look at the first three.

* **TIP:** Read HTML gives you a *list of all of the HTML tables on the page*. This means "the output is going to look ugly, but it's not a Series, it's a list of dataframes."
* **TIP:** I think you should save that as a variable called `tables`
* **TIP:** Then just keep looking at the first table, the second, the third, with `[0]`, `[1]` etc, until you find the one you want. Then save that as `df`.